In [1]:
from pyspark.sql import SparkSession

In [2]:
spark.stop()  # Stop the existing session

In [3]:
# Create a Spark session with Iceberg configurations
spark = SparkSession.builder \
    .appName("Iceberg Table Creation") \
    .config("spark.sql.catalog.my_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.my_catalog.type", "rest") \
    .config("spark.sql.catalog.my_catalog.uri", "http://iceberg-rest:8181") \
    .getOrCreate()

24/11/14 11:47:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
import pyarrow.parquet as pq
import pyarrow as pa

# Define paths
input_path = "/home/iceberg/silver_layer/json_to_parquet.parquet"
output_path = "/home/iceberg/silver_layer/json_to_parquet_micro.parquet"

# Read the Parquet file with pyarrow
table = pq.read_table(input_path)
table

pyarrow.Table
Id: string
Timestamp: timestamp[ns]
ReferenceId: string
AppIdentificator: string
PaymentType: string
PaymentProvider: string
Country: string
TenantId: string
Amount: double
Units: int64
TotalAmount: double
Payload: string
----
Id: [["21be5e99-2b93-41b8-8bb5-868ad323b430","9be186c2-2629-4586-9169-f412e6e752f1","cac6753f-7041-459b-a3b1-06ca40a890af","d8423bfa-3216-4d3f-81a4-c77e7c054f90","8e019a7e-bcc7-4db1-baff-d53e3dda9821",...,"ba21f33f-d170-49aa-b10e-691065bdba00","086ec9e6-77ad-4a7e-a892-1f0d6ba211b0","d88b644a-a23b-4879-b402-f82465db9973","f8b0c514-1251-429c-b511-b6724b57e994","80e8ea05-4884-4887-bf93-1f4f1c21b955"]]
Timestamp: [[2023-06-13 07:56:15.000000000,2023-06-23 06:04:27.000000000,2022-10-26 06:11:29.000000000,2020-01-20 17:57:30.000000000,2021-09-20 20:34:36.000000000,...,2020-12-15 09:30:03.000000000,2021-08-15 07:04:01.000000000,2023-01-20 03:51:21.000000000,2020-07-21 06:23:22.000000000,2020-09-28 07:28:44.000000000]]
ReferenceId: [["5ad382ce-6d21-46c9-b03

In [5]:
# Convert to microsecond precision and save as a new Parquet file
pq.write_table(table, output_path, coerce_timestamps='us')
table1 = pq.read_table(output_path)
table1

pyarrow.Table
Id: string
Timestamp: timestamp[us]
ReferenceId: string
AppIdentificator: string
PaymentType: string
PaymentProvider: string
Country: string
TenantId: string
Amount: double
Units: int64
TotalAmount: double
Payload: string
----
Id: [["21be5e99-2b93-41b8-8bb5-868ad323b430","9be186c2-2629-4586-9169-f412e6e752f1","cac6753f-7041-459b-a3b1-06ca40a890af","d8423bfa-3216-4d3f-81a4-c77e7c054f90","8e019a7e-bcc7-4db1-baff-d53e3dda9821",...,"ba21f33f-d170-49aa-b10e-691065bdba00","086ec9e6-77ad-4a7e-a892-1f0d6ba211b0","d88b644a-a23b-4879-b402-f82465db9973","f8b0c514-1251-429c-b511-b6724b57e994","80e8ea05-4884-4887-bf93-1f4f1c21b955"]]
Timestamp: [[2023-06-13 07:56:15.000000,2023-06-23 06:04:27.000000,2022-10-26 06:11:29.000000,2020-01-20 17:57:30.000000,2021-09-20 20:34:36.000000,...,2020-12-15 09:30:03.000000,2021-08-15 07:04:01.000000,2023-01-20 03:51:21.000000,2020-07-21 06:23:22.000000,2020-09-28 07:28:44.000000]]
ReferenceId: [["5ad382ce-6d21-46c9-b03e-ef71fab2d672","f6240a7a-81a1

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyarrow.parquet as pq
import pyarrow as pa

In [7]:
# Step 1: Load the Parquet file into Spark DataFrame
df = spark.read.parquet(output_path)
# Convert to Pandas DataFrame
df_pandas = df.toPandas()
df_pandas

Id           Timestamp  \
0    21be5e99-2b93-41b8-8bb5-868ad323b430 2023-06-13 07:56:15   
1    9be186c2-2629-4586-9169-f412e6e752f1 2023-06-23 06:04:27   
2    cac6753f-7041-459b-a3b1-06ca40a890af 2022-10-26 06:11:29   
3    d8423bfa-3216-4d3f-81a4-c77e7c054f90 2020-01-20 17:57:30   
4    8e019a7e-bcc7-4db1-baff-d53e3dda9821 2021-09-20 20:34:36   
..                                    ...                 ...   
995  ba21f33f-d170-49aa-b10e-691065bdba00 2020-12-15 09:30:03   
996  086ec9e6-77ad-4a7e-a892-1f0d6ba211b0 2021-08-15 07:04:01   
997  d88b644a-a23b-4879-b402-f82465db9973 2023-01-20 03:51:21   
998  f8b0c514-1251-429c-b511-b6724b57e994 2020-07-21 06:23:22   
999  80e8ea05-4884-4887-bf93-1f4f1c21b955 2020-09-28 07:28:44   

                              ReferenceId  \
0    5ad382ce-6d21-46c9-b03e-ef71fab2d672   
1    f6240a7a-81a1-474d-99ba-824fc5d85a70   
2    b79dd28d-0d87-4ffb-b492-63625ca381c2   
3    4bad134f-cbb5-4d5c-a27d-4fbc78bd205b   
4    64e58647-9aa7-4090-9707-6b9a7a724c40   
..                                    ...   
995  f73b83b3-602a-4a70-812b-a7699e27bbfb   
996  d74bd295-9820-4942-9c4d-dff6a842d6c7   
997  1932327a-aa20-4a5a-9a58-fad781f907c4   
998  dbb20852-15df-4745-93c7-8f0725f043ed   
999  a5d40aa9-050b-4d8a-95a7-8d3ba847a7eb   

                         AppIdentificator            PaymentType  \
0    dbded140-e0d6-4ed3-8764-224aa5c1e687              Apple Pay   
1    028e6189-36be-43a2-9399-fa8a20e56068  Direct Debit Mandates   
2    45712b6a-0eb0-46c3-9824-601eed84bb27  Direct Debit Mandates   
3    fe4e1c54-e4df-457b-9aad-f985498ec83d          Bank Transfer   
4    a0e559ef-6cf0-4f8a-85ee-baef6f152427  Direct Debit Mandates   
..                                    ...                    ...   
995  b6c7817f-561d-4e87-a73f-3e1351de7496           Momo Wallets   
996  dbded140-e0d6-4ed3-8764-224aa5c1e687           Momo Wallets   
997  45712b6a-0eb0-46c3-9824-601eed84bb27          Bank Transfer   
998  4505a8a2-7f93-47bd-8347-97dc1415a1bb           Momo Wallets   
999  f5605f71-3e82-440e-83bd-9f44d09f977c             Google Pay   

    PaymentProvider       Country  TenantId  Amount  Units  TotalAmount  \
0             GTpay         Kenya  tenant_3  416.53      4       868.62   
1          Paystack      Ethiopia  tenant_8  173.20      4       480.26   
2             GTpay        Rwanda  tenant_5  430.19     10       843.33   
3       Flutterwave        Rwanda  tenant_9  320.27      8       221.53   
4          MonoMomo        Rwanda  tenant_3  537.98      8       530.97   
..              ...           ...       ...     ...    ...          ...   
995     Flutterwave  South Africa  tenant_2  425.81     10       572.11   
996        Paystack         Kenya  tenant_9  749.79      3       875.45   
997     Interswitch       Senegal  tenant_5  339.14      1       775.61   
998     Flutterwave         Ghana  tenant_3  721.55      8       629.04   
999        Paystack       Nigeria  tenant_7  775.79      9       143.66   

                                               Payload  
0    Key town population trip forget defense agreem...  
1    Significant seem produce. Manager for analysis...  
2            Blue why off alone consider walk pattern.  
3    Guy cut see need.\nContinue our send another i...  
4    Beat water medical service.\nDevelop whatever ...  
..                                                 ...  
995    I employee away cultural once. No cover answer.  
996  Ahead difference skill herself type many. Requ...  
997  Will thing environment picture Congress. Stree...  
998  Everything sure TV eight test. Entire discover...  
999  Cut test test increase line help. Probably lik...  

[1000 rows x 12 columns]

In [8]:
# Step 2: Create Iceberg table if it doesn't exist already
# Define the table name and schema for Iceberg table
table_name = "my_catalog.default.json_to_iceberg_table"

In [9]:
# Check if the table exists and create if it doesn't
spark.sql(f"""
  CREATE TABLE IF NOT EXISTS {table_name} (
    Id STRING,
    Timestamp TIMESTAMP,
    ReferenceId STRING,
    AppIdentificator STRING,
    PaymentType STRING,
    PaymentProvider STRING,
    Country STRING,
    TenantId STRING,
    Amount DOUBLE,
    Units INT,
    TotalAmount DOUBLE,
    Payload STRING
  )
  USING iceberg
  PARTITIONED BY (days(Timestamp))  -- Partitioning by day based on Timestamp
""")

DataFrame[]

In [10]:
spark.sql(f"DESCRIBE {table_name}").show()


+----------------+---------------+-------+
|        col_name|      data_type|comment|
+----------------+---------------+-------+
|              Id|         string|   NULL|
|       Timestamp|      timestamp|   NULL|
|     ReferenceId|         string|   NULL|
|AppIdentificator|         string|   NULL|
|     PaymentType|         string|   NULL|
| PaymentProvider|         string|   NULL|
|         Country|         string|   NULL|
|        TenantId|         string|   NULL|
|          Amount|         double|   NULL|
|           Units|            int|   NULL|
|     TotalAmount|         double|   NULL|
|         Payload|         string|   NULL|
|                |               |       |
|  # Partitioning|               |       |
|          Part 0|days(Timestamp)|       |
+----------------+---------------+-------+



In [11]:
df.write \
    .format("iceberg") \
    .option("parquet.compression", "SNAPPY") \
    .mode("append") \
    .save(table_name)


In [12]:
spark.sql("SELECT * FROM my_catalog.default.json_to_iceberg_table WHERE PaymentType = 'Card'").show()

+--------------------+-------------------+--------------------+--------------------+-----------+---------------+------------+---------+------+-----+-----------+--------------------+
|                  Id|          Timestamp|         ReferenceId|    AppIdentificator|PaymentType|PaymentProvider|     Country| TenantId|Amount|Units|TotalAmount|             Payload|
+--------------------+-------------------+--------------------+--------------------+-----------+---------------+------------+---------+------+-----+-----------+--------------------+
|e10e7c4d-7fb6-49c...|2022-07-11 03:15:48|2b5f8d02-e1c0-479...|41f1a844-dcac-477...|       Card|    Interswitch|     Senegal| tenant_6| 578.3|   10|     858.34|Bring necessary r...|
|49c8c616-d528-4b7...|2022-07-12 10:44:51|caab6ad2-3a99-40c...|fe4e1c54-e4df-457...|       Card|    Interswitch|    Zimbabwe|tenant_10|273.57|    8|     620.36|Would various hot...|
|1c72beeb-8324-487...|2022-07-15 03:23:52|7576ee46-6c25-460...|45712b6a-0eb0-46c...|      

In [13]:
spark.sql("SHOW TABLES IN my_catalog.default").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|json_to_iceberg_t...|      false|
+---------+--------------------+-----------+



In [14]:
spark.sql("SELECT COUNT(*) AS row_count FROM my_catalog.default.json_to_iceberg_table").show()

+---------+
|row_count|
+---------+
|     1000|
+---------+

